In [1]:
#import h5py
import pickle
import time
import numpy as np
#import pandas as pd
#import wavelets as wl
#from scipy import signal
from matplotlib import pyplot as plt

#### Loading spikes

In [3]:
npys_dir = '/home/maspe/filer/SERT/ALL/npys/'
IDs_WT = ['SERT1597'] #, 'SERT1659', 'SERT1678', 'SERT1908', 'SERT1984', 'SERT1985', 'SERT2014']

In [ ]:
sample_rate = 30000
all_mice_WT = {}

for ID in IDs_WT:
    print('Importing {}...'.format(ID))
    clock = time.time()
    
    # Get the raw data, sampled at 30 kHz
    data = np.load(npys_dir + ID + '_spikes_WT.npy', allow_pickle=True)
    
    # Reduce it to a sample rate of 1 Hz
    all_mice_WT[ID] = np.add.reduceat(data, range(0, data.shape[1], sample_rate), axis=1)
        
    print('{} imported in {} min.'.format(ID, (time.time() - clock) / 60))

    
print('Saving the data...')
with open(npys_dir + 'reduced_WT.spikes', 'wb') as f:
    pickle.dump(all_mice_WT, f, protocol=2)
print('Done!')

In [ ]:
data.shape

####################

In [5]:
with open(npys_dir + 'reduced_WT.spikes', 'rb') as f:
    reduced_spikes = pickle.load(f) #, encoding='latin1')

In [ ]:
np.mean(mice_matrix[:, :120], axis=1).shape

In [13]:
start_OF = 120 # After first 2 minutes at 1 Hz

normalised_spikes = {}
for mouse in reduced_spikes.keys():
    mice_matrix = reduced_spikes[mouse]
    
    n_units = mice_matrix.shape[0]
    
    #baseline_mean = np.repeat(np.mean(mice_matrix[:, :start_OF], axis=1), start_OF)

    # Create baseline normalised by the mean
    # Solution from: https://stackoverflow.com/questions/26333005/numpy-subtract-every-row-of-matrix-by-vector/26333184
    baseline = (mice_matrix[:, :start_OF].T - np.mean(mice_matrix[:, :start_OF], axis=1)).T
    OF = (mice_matrix[:, start_OF:].T - np.mean(mice_matrix[:, start_OF:], axis=1)).T 
    
    normalised_spikes[mouse] = np.concatenate([baseline, OF], axis=1)
    

In [ ]:
np.mean(mice_matrix[:, :start_OF], axis=1).shape

In [14]:
normalised_spikes

{'SERT1597': array([[186.83333333, 179.83333333, 148.83333333, ..., -26.5965812 ,
         -26.5965812 , -69.5965812 ],
        [198.625     , 180.625     , 154.625     , ..., -20.4034188 ,
         -33.4034188 , -68.4034188 ],
        [200.475     , 178.475     , 159.475     , ..., -29.42051282,
         -32.42051282, -74.42051282],
        ...,
        [-23.25833333, -47.25833333, -58.25833333, ..., -48.66837607,
         327.33162393, 112.33162393],
        [-23.325     , -49.325     , -62.325     , ..., -41.85470085,
         344.14529915, 119.14529915],
        [-25.55833333, -44.55833333, -60.55833333, ..., -51.91623932,
         326.08376068, 115.08376068]])}

In [15]:
reduced_spikes

{'SERT1597': array([[319., 312., 281., ...,  56.,  56.,  13.],
        [335., 317., 291., ...,  66.,  53.,  18.],
        [341., 319., 300., ...,  59.,  56.,  14.],
        ...,
        [ 57.,  33.,  22., ..., 156., 532., 317.],
        [ 56.,  30.,  17., ..., 163., 549., 324.],
        [ 56.,  37.,  21., ..., 154., 532., 321.]])}

In [ ]:
### Extract a windows of n_secs length at the beginning and end of the task
n_min = 2
sample_rate = 30000
start_task = n_min * 60 * sample_rate

#window = int(sampleRate * secs)
n_points = 30000 * 60 * 10 # Length of the OF
window = sample_rate * n_min * 60 # 2 min windows
exclude_window = sample_rate * exclude_sec # 15 sec windows to exclude at the HC-OF transition

In [ ]:
with open('/home/maspe/filer/SERT/ALL/npys/channels_by_spikes_WT.info', 'rb') as f:
    all_channels_WT = pickle.load(f)

In [ ]:
mPFC_WT = {}
NAC_WT  = {}
BLA_WT  = {}
vHip_WT = {}

for mouse in all_mice_WT.keys():
    mPFC_WT_indexes  = [i for i,x in enumerate(all_channels_WT[mouse]) if x == 'mPFC_left']
    NAC_WT_indexes = [i for i,x in enumerate(all_channels_WT[mouse]) if x == 'NAC_left']
    BLA_WT_indexes  = [i for i,x in enumerate(all_channels_WT[mouse]) if x == 'BLA_left']
    vHip_WT_indexes  = [i for i,x in enumerate(all_channels_WT[mouse]) if x == 'vHipp_left']
    


In [ ]:
    
    mPFC_WT[mouse] = np.sum(all_mice_WT[mouse][mPFC_WT_indexes, :], axis=0)
    NAC_WT[mouse] = np.sum(all_mice_WT[mouse][NAC_WT_indexes, :], axis=0)
    BLA_WT[mouse] = np.sum(all_mice_WT[mouse][BLA_WT_indexes, :], axis=0)
    vHip_WT[mouse] = np.sum(all_mice_WT[mouse][vHip_WT_indexes, :], axis=0)

In [ ]:
npys_dir = '/home/maspe/filer/SERT/ALL/npys/'

all_spikes_WT = {'mPFC': mPFC_WT, 'BLA': BLA_WT, 'NAC': NAC_WT, 'vHip': vHip_WT}
np.save('/home/maspe/filer/SERT/ALL/npys/all_spikes_WT', all_spikes_WT)

In [ ]:
factor = 15000
structure = 'vHip'

iteration = 0
t = np.zeros((len(all_spikes_WT[structure].keys()), 7200000 / factor))
for mouse in all_spikes_WT[structure].keys():
    t[iteration,:]=np.add.reduceat(all_spikes_WT[structure][mouse],
                                   range(0, all_spikes_WT[structure][mouse].shape[0], factor))
    
    iteration += 1
    
tmean_WT=np.mean(t, axis=0)
tsem_WT=np.std(t, axis=0) / np.sqrt(t.shape[0])

iteration = 0
t = np.zeros((len(all_spikes_KO[structure].keys()), 7200000 / factor))
for mouse in all_spikes_KO[structure].keys():
    t[iteration,:]=np.add.reduceat(all_spikes_KO[structure][mouse],
                                   range(0, all_spikes_KO[structure][mouse].shape[0], factor))
    
    iteration += 1
    
tmean_KO=np.mean(t, axis=0)
tsem_KO=np.std(t, axis=0) / np.sqrt(t.shape[0])

tx = range(0, tmean_KO.shape[0])


### Plot
#plt.figure(figsize=(20,10))
#plt.subplot(2,2,1)

f, (ax, ax2) = plt.subplots(1, 2, sharey=True)
ax.plot(tx, tmean_WT, '-', color='blue', alpha=1)
ax.fill_between(tx, tmean_WT + tsem_WT, tmean_WT - tsem_WT, alpha=0.5)

ax.plot(tx, tmean_KO, '-', color='red', alpha=1)
ax.fill_between(tx, tmean_KO + tsem_KO, tmean_KO - tsem_KO, alpha=0.5)

ax2.plot(tx, tmean_WT, '-', color='blue', alpha=1)
ax2.fill_between(tx, tmean_WT + tsem_WT, tmean_WT - tsem_WT, alpha=0.5)

ax2.plot(tx, tmean_KO, '-', color='red', alpha=1)
ax2.fill_between(tx, tmean_KO + tsem_KO, tmean_KO - tsem_KO, alpha=0.5)

ax.set_xlim(0, 240)  # outliers only
ax2.set_xlim(240, 480)  # most of the data

ax.set_ylim(0, 2500)  # outliers only


# hide the spines between ax and ax2
ax.spines['right'].set_visible(False)
ax2.spines['left'].set_visible(False)
ax.yaxis.tick_left()
ax.tick_params(labelright='off')
ax2.yaxis.tick_right()

d = .02 # how big to make the diagonal lines in axes coordinates
# arguments to pass plot, just so we don't keep repeating them
kwargs = dict(transform=ax.transAxes, color='k', clip_on=False)
ax.plot((1-d,1+d), (-d,+d), **kwargs)
ax.plot((1-d,1+d),(1-d,1+d), **kwargs)

kwargs.update(transform=ax2.transAxes)  # switch to the bottom axes
ax2.plot((-d,+d), (1-d,1+d), **kwargs)
ax2.plot((-d,+d), (-d,+d), **kwargs)

ax.set_xticklabels(['', '1', '2'], fontsize=14)
ax2.set_xticklabels(['7', '8', '9', '10'])

ax.tick_params(labelsize=14)
ax2.tick_params(labelsize=14)


plt.text(380, 2200, structure, fontsize=18)

plt.savefig('/home/maspe/filer/SERT/ALL/figs/' + structure + '_perispikes.pdf', dpi=150, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format='pdf', transparent=False)


In [ ]:




# NAC
iteration = 0
t = np.zeros((len(NAC_WT.keys()), window*2 / 60))
for mouse in NAC_WT.keys():
    t[iteration,:]=np.add.reduceat(NAC_WT[mouse], range(0, NAC_WT[mouse].shape[0], 60))
    
    iteration += 1
    
tmean_WT=np.mean(t, axis=0)

iteration = 0
t = np.zeros((len(NAC_KO.keys()), window*2 / 60))
for mouse in NAC_KO.keys():
    t[iteration,:]=np.add.reduceat(NAC_KO[mouse], range(0, NAC_KO[mouse].shape[0], 60))
    
    iteration += 1
    
tmean_KO=np.mean(t, axis=0)

plt.subplot(2,2,2)
plt.plot(tx, tmean_WT, '-', color='blue', alpha=0.7)
plt.plot(tx, tmean_KO, '-', color='red', alpha=0.7)
plt.xticks([0, 20000, 40000, 60000, 80000, 100000, 120000], ['-3', '-2', '-1', '0', '1', '2', '3'], fontsize=16)
plt.yticks(fontsize=16)
plt.ylim([0,70])
plt.axvline(x=60000, color='black')
#plt.xlabel('time (s)')
#plt.ylabel('units')
plt.title('NAC', fontsize=22)


# BLA
iteration = 0
t = np.zeros((len(BLA_WT.keys()), window*2 / 60))
for mouse in BLA_WT.keys():
    t[iteration,:]=np.add.reduceat(BLA_WT[mouse], range(0, BLA_WT[mouse].shape[0], 60))
    
    iteration += 1
    
tmean_WT=np.mean(t, axis=0)

iteration = 0
t = np.zeros((len(BLA_KO.keys()), window*2 / 60))
for mouse in BLA_KO.keys():
    t[iteration,:]=np.add.reduceat(BLA_KO[mouse], range(0, BLA_KO[mouse].shape[0], 60))
    
    iteration += 1
    
tmean_KO=np.mean(t, axis=0)

plt.subplot(2,2,3)
plt.plot(tx, tmean_WT, '-', color='blue', alpha=0.7)
plt.plot(tx, tmean_KO, '-', color='red', alpha=0.7)
plt.xticks([0, 20000, 40000, 60000, 80000, 100000, 120000], ['-3', '-2', '-1', '0', '1', '2', '3'], fontsize=16)
plt.yticks(fontsize=16)
plt.ylim([0,70])
plt.axvline(x=60000, color='black')
plt.xlabel('time (s)', fontsize=18)
plt.ylabel('units', fontsize=18)
plt.title('BLA', fontsize=24)


# vHip
iteration = 0
t = np.zeros((len(vHip_WT.keys()), window*2 / 60))
for mouse in vHip_WT.keys():
    t[iteration,:]=np.add.reduceat(vHip_WT[mouse], range(0, vHip_WT[mouse].shape[0], 60))
    
    iteration += 1
    
tmean_WT=np.mean(t, axis=0)

iteration = 0
t = np.zeros((len(vHip_KO.keys()), window*2 / 60))
for mouse in vHip_KO.keys():
    t[iteration,:]=np.add.reduceat(vHip_KO[mouse], range(0, vHip_KO[mouse].shape[0], 60))
    
    iteration += 1
    
tmean_KO=np.mean(t, axis=0)

plt.subplot(2,2,4)
plt.plot(tx, tmean_WT, '-', color='blue', alpha=0.7)
plt.plot(tx, tmean_KO, '-', color='red', alpha=0.7)
plt.xticks([0, 20000, 40000, 60000, 80000, 100000, 120000], ['-3', '-2', '-1', '0', '1', '2', '3'], fontsize=16)
plt.yticks(fontsize=16)
plt.ylim([0,70])
plt.axvline(x=60000, color='black')
plt.xlabel('time (s)', fontsize=18)
#plt.ylabel('units')
plt.title('vHip', fontsize=24)


plt.savefig('/home/maspe/filer/SERT/ALL/figs/perispikes.png', dpi=150, facecolor='w', edgecolor='w',
            orientation='portrait', papertype=None, format='png', transparent=False)
    
#plt.close()

#####################################

#### Backup code

In [ ]:
spikes = []
for unit in range(len(spikes_epochs_pre)):
    # np.isin(element, test_elements)
    spikes.append(np.isin(epoch_matrix, [spikes_epochs_pre[unit] + spikes_epochs_post[unit]]))

In [ ]:
spikes_epochs_pre = []
spikes_epochs_post = []
for unit in range(len(all_units)):
    spiketimes_pre = []
    spiketimes_post = []
    for epoch in range(n_epochs):
        spiketimes_pre.extend(all_units[unit][(all_units[unit] > epochs_pre[epoch][0]) & (all_units[unit] < epochs_post[epoch][1])])
        spiketimes_post.extend(all_units[unit][(all_units[unit] > epochs_post[epoch][0]) & (all_units[unit] < epochs_post[epoch][1])])
    spikes_epochs_pre.append(spiketimes_pre)
    spikes_epochs_post.append(spiketimes_post)

In [ ]:
len(spikes_epochs_pre[52])

In [ ]:
spikes = []
for unit in range(len(spikes_epochs_pre)):
    # np.isin(element, test_elements)
    spikes.append(np.isin(epoch_matrix, [spikes_epochs_pre[unit] + spikes_epochs_post[unit]]))

In [ ]:
np.sum(spikes[50], axis=0).shape

In [ ]:
### Collect the name of channels, by mouse
#all_units_WT = {}
all_channels_WT = {}

n_channels = 32
for mouse in all_mice_WT.keys():

    units = []
    channels_id = []
    
    iteration = 0
    for channel in range(n_channels):
        for unit in all_mice_WT[mouse][channel]['spiketimes'].keys():
            #units.append(all_mice_WT[mouse][channel]['spiketimes'][unit][()]) # Final "[()]" is to import values from h5py 
      
            channels_id.append(all_info_WT[mouse]['channels_locs'][iteration])
        
        iteration += 1
        
            
    #all_units_WT[mouse] = units
    all_channels_WT[mouse] = channels_id
        
# print('n units = %i' %len(all_units))

### Same for KOs
#all_units_KO = {}
all_channels_KO = {}
for mouse in all_mice_KO.keys():
    n_channels = 32
    units = []
    channels_id = []
    
    iteration = 0
    for channel in range(n_channels):
        for unit in all_mice_KO[mouse][channel]['spiketimes'].keys():
            #units.append(all_mice_KO[mouse][channel]['spiketimes'][unit][()]) # Final "[()]" is to import values from h5py 
      
            channels_id.append(all_info_KO[mouse]['channels_locs'][iteration])
        
        iteration += 1
        
            
    #all_units_KO[mouse] = units
    all_channels_KO[mouse] = channels_id
        
# print('n units = %i' %len(all_units))

